In [1]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import col, to_date
import pyspark.sql.functions as F

In [2]:
spark = SparkSession.builder.appName("UID Event").getOrCreate()

25/04/07 11:52:35 WARN Utils: Your hostname, Les-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.202 instead (on interface en0)
25/04/07 11:52:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/07 11:52:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark.sparkContext._gateway.jvm.scala.util.Properties.versionString()

'version 2.12.18'

In [27]:
# Define the data
data = [
    {
        "uid": "user1",
        "data_date": "2022-01-01",
        "event_type": "insert",
        "value": "value",
    },
    {
        "uid": "user1",
        "data_date": "2022-01-02",
        "event_type": "update",
        "value": "value_1",
    },
    {
        "uid": "user2",
        "data_date": "2022-01-03",
        "event_type": "update",
        "value": "value_2",
    },
    {
        "uid": "user1",
        "data_date": "2022-01-04",
        "event_type": "insert",
        "value": "value_3",
    },
    {
        "uid": "user3",
        "data_date": "2022-01-05",
        "event_type": "update",
        "value": "value_3",
    },
]

# Convert the data to a PySpark DataFrame
df = spark.createDataFrame(data)
df.schema

StructType([StructField('data_date', StringType(), True), StructField('event_type', StringType(), True), StructField('uid', StringType(), True), StructField('value', StringType(), True)])

The new schema gonna have:
- `uid`: id to check change
- `data_date`: I thought that was the snapshot
- `value`: Current value (in the value date)
- `from_date`: Active date
- `to_date`: Outdate date
- `snapshot_date`: occurs on which snapshot

In [46]:
def init_load_scd2(cdc_df: DataFrame, data_date: str) -> DataFrame:
    """
    Filter the whole crap of the cdc_df, ranked by status

    What is latest then it gonna be the o
    """
    cdc_df.createOrReplaceTempView("cdc")
    current = spark.sql(f"""
        with ranked as (
            select
                *,
                rank(data_date) over (partition by uid order by data_date desc) as rk
            from cdc
            where data_date <= '{data_date}'
        )
        select * from ranked where rk = 1
    """).drop("rk")
    return (
        current.withColumn("snapshot_date", F.lit(data_date))
        .withColumn("from_date", F.lit(data_date))
        .withColumn("to_date", F.lit("3000-12-31"))
        .withColumn("status", F.lit(1))
    )


def incremental_load_scd2(
    current: DataFrame, cdc_df: DataFrame, data_date: str
) -> DataFrame:
    """
    Flow:
    - with insert: auto insert, with the `from_date`=current data date and `to_date`='3000-12-31'
    - with update:
    """

    ...


def backfill_run_scd2(data_date: str) -> DataFrame: ...

In [47]:
# expected the initial load to the group by
current_df = init_load_scd2(df, "2022-01-04")
current_df.show()

+----------+----------+-----+-------+-------------+----------+----------+------+
| data_date|event_type|  uid|  value|snapshot_date| from_date|   to_date|status|
+----------+----------+-----+-------+-------------+----------+----------+------+
|2022-01-04|    insert|user1|value_3|   2022-01-04|2022-01-04|3000-12-31|     1|
|2022-01-03|    update|user2|value_2|   2022-01-04|2022-01-04|3000-12-31|     1|
+----------+----------+-----+-------+-------------+----------+----------+------+

